# Code Sample
Shown here is a Jupyter notebook sample of the example provided in this repository. This is meant to provide a demonstration of the expected behavior/output

In [ ]:
# miscellaneous imports and configs for running examples in docs
import dask.array as da
from rich import inspect
import pathlib
import sys
from jupyterthemes import get_themes
import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('chesterish')

### How Submit a Function to a SLURM Cluster Without Using a Decorator

In [ ]:
sys.path.append('../../')
from pyslurm_decorators import SLURMClusterCallback
from pyslurm_decorators.schemas.slurm import Options

# define the cluster options
options = Options(cores=2,
    n_workers=2,
    processes=1, 
    memory="5GB",
    shebang='#!/usr/bin/env bash',
    queue="normal",
    walltime="00:30:00",
    local_directory='/tmp',
    death_timeout="15s",
    log_directory=f"{str(pathlib.Path.home())}/dask-test",
    project="boc",
    show_progress=True)

# define a function you want submitted to the cluster
def calc_pi_mc(size_in_bytes, chunksize_in_bytes=200e6):
    """Calculate PI using a Monte Carlo estimate."""
    
    size = int(size_in_bytes / 8)
    chunksize = int(chunksize_in_bytes / 8)
    
    xy = da.random.uniform(0, 1,
                           size=(size / 2, 2),
                           chunks=(chunksize / 2, 2))
    
    in_circle = ((xy ** 2).sum(axis=-1) < 1)
    pi = 4 * in_circle.mean()

    return pi

# instantiate the cluster object
cluster = SLURMClusterCallback(callback=calc_pi_mc, options=options)

# take a look at the object: this is what is returned when calling a @slurm decorated function
inspect(cluster, methods='all')

# submit the function the cluster and return the result
result = cluster.submit(size_in_bytes=1e9)
print("My Pi Estimate is: ", result)